In [ ]:
import util.console

In [1]:
from util.download_files import main
main()

	- resumes/Alex_Chavez_49158527.pdf complete
	- resumes/Angel Moreno.docx complete
	- resumes/April Powell.docx complete
	- resumes/Billy Perkins.docx complete
	- resumes/Anthony Smith.docx complete
	- resumes/Ankita Bhargava.docx complete
	- resumes/Brianna Luter.docx complete
	- resumes/Amadeus Martinez.docx complete
	- resumes/Allen_Goldner_49910588.pdf complete
	- resumes/Allison Hall.docx complete
	- resumes/Alonzo_Hoyos_50495262.pdf complete
	- resumes/Ayomide Otukoya.docx complete
	- resumes/Austin Jackson.docx complete
	- resumes/Bill_Chapman_50409201.pdf complete
	- resumes/Alicia_Valencia_49980384.pdf complete
	- resumes/Annette_Burnett_46460051.pdf complete
	- resumes/Beau_Greene_50161629.pdf complete
	- resumes/Amir_Khawaja_49945524.pdf complete
	- resumes/Amanda_Walters_49534632.pdf complete
	- resumes/Bob_van_Orden_47437954.pdf complete
	- resumes/Aviad_Inger_49238931.pdf complete
	- resumes/Bruce Carl Guercio.docx complete
	- resumes/Andrea_Yankovich_50130077.pdf complet

In [ ]:
from util.text_extractor import TextExtractor

extractor = TextExtractor(use_linux_support=True, persist_file="../cache.json")

In [ ]:
import json
names = [json.loads(value)["candidate_name"] for value in extractor._persisted_disk_cache.values()]
names.sort()

In [ ]:
names

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import os

resume_paths = [os.path.join(root, f) for root, __, files in os.walk("../resumes") for f in files][:100]
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    results = [y for x in executor.map(extractor.extract, resume_paths) for y in x]
    results.sort(key=lambda x : x.candidate_name)
    results = [x for x in results if x.candidate_name != 'None']
    print("Done.")



In [ ]:
extractor.persist()

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

from architecture.resume_agent import ResumeAgent

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.0)

resume_agents = [
    ResumeAgent(document=doc) for doc in results
]

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(model='gpt-4o-mini', temperature=0.0)

context = """
You are a job recruiter. You have all the resumes and information about candidates. Whenever you use pagination, return the results as is, and please notify the user that the results shown have been limited to reduce API spending, and if they want to show more, they can change the value of the `page` argument.
"""

react = ReActAgent.from_tools([
    FunctionTool.from_defaults(ResumeAgent.get_candidate_names),
    FunctionTool.from_defaults(ResumeAgent.get_total_pages),
    FunctionTool.from_defaults(ResumeAgent.prompt_one),
    FunctionTool.from_defaults(ResumeAgent.prompt_many),
    FunctionTool.from_defaults(ResumeAgent.get_resume_of_many),
    FunctionTool.from_defaults(ResumeAgent.get_resume_of_one),
], llm=Settings.llm, verbose=True, context=context, max_iterations=20)

In [7]:
react.chat("Can you list all candidates who have at least 2 years of door-to-door sales experience?")